In [1]:
import itertools
import pandas as pd
import numpy as np
import csv

In [2]:
file_path = './data/newtest01.xtb.tbw'
dic_path = './data/CensusMainShortForm.dcf'

f = open(file_path, 'r')


In [3]:
dimension = []
all_values = []
all_codes= []
all_rows = []
values = []
area_names = []
area_codes = []

In [4]:
i=1
notFirst=False
while True:
        
        line = f.readline()
        if not line:
            if notFirst:
                all_values.append(values)
            break
        else:
            if line.find('[Variable]') != -1:
                if notFirst:
                    all_values.append(values)
                name = f.readline()
                name = name.replace("\n", "")
                name =  name[5:]
                dimension.append(name)
                values = []
                notFirst = True
            if line.find('[Value]') != -1:
                label = f.readline()
                label = label.replace("\n", "")
                label = label[6:]
                #print(label)
                values.append(label)
            if line.find('AreaName') == 0: 
                area = line[9:]
                #print(area)
                area_names.append(area.strip())
            if line.find('BreakKey') == 0: 
                code = line[9:]
                code = code.replace(';', '')
                #print(area)
                area_codes.append(code.strip())
            if line.find('Row') == 0: 
                #row = f.readline()
                row = line[4:]
                i = i+1
                arr = [ x.strip() for x in row.split('  ')]
                all_rows += arr
              

               

In [5]:
len(all_rows)

18864

In [6]:
#print(area_codes)              
#all_values[0] = area_codes                
#print(dimension)
#print(area_names)
#print(all_values)
#print(all_rows)

In [7]:
print(all_values[0])

['%AreaName%']


In [8]:
d = open(dic_path, 'r', encoding='utf-8')
header =['cod','desc', 'cod_list']
dim_values = []
total = []
a =  np.empty((0, 3))

def getCodes(d, dim):
    d.seek(0)
    values = []
    dim=dim + "\n"
    while True:
        line = d.readline()
        if not line:
            break
        if line.find(dim) != -1:
            line = d.readline()
            while line.find('Value') != -1:
                line = line.replace("\n", "")
                line = line[6:]
                line = line + ';' + dim
                item = line.split(';')
                values.append(item)
                line = d.readline()
                if line.find('Note=') != -1:
                    line = d.readline()
    return values


for dim in dimension[1:]:
    values = getCodes(d, dim)
    total = ['0', 'Total', dim]
    values.append(total)
    print(values)
    print("\n")
    a = np.concatenate((a, values), axis=0)

[['3:5', '3 - 5', 'P12_VS4\n'], ['6:9', '6 - 9', 'P12_VS4\n'], ['10:14', '10 - 14', 'P12_VS4\n'], ['15:24', '15 - 24', 'P12_VS4\n'], ['25:34', '25 - 34', 'P12_VS4\n'], ['35:44', '35 - 44', 'P12_VS4\n'], ['45:54', '45 - 54', 'P12_VS4\n'], ['55:64', '55 - 64', 'P12_VS4\n'], ['65:74', '65 - 74', 'P12_VS4\n'], ['75:120', '75 - 120', 'P12_VS4\n'], ["'   '", 'Notappl', 'P12_VS4\n'], ['0', 'Total', 'P12_VS4']]


[['1', 'Male', 'P11_VS1\n'], ['2', 'Female', 'P11_VS1\n'], ['3', 'Intersex', 'P11_VS1\n'], ['0', 'Total', 'P11_VS1']]




In [9]:
a

array([['3:5', '3 - 5', 'P12_VS4\n'],
       ['6:9', '6 - 9', 'P12_VS4\n'],
       ['10:14', '10 - 14', 'P12_VS4\n'],
       ['15:24', '15 - 24', 'P12_VS4\n'],
       ['25:34', '25 - 34', 'P12_VS4\n'],
       ['35:44', '35 - 44', 'P12_VS4\n'],
       ['45:54', '45 - 54', 'P12_VS4\n'],
       ['55:64', '55 - 64', 'P12_VS4\n'],
       ['65:74', '65 - 74', 'P12_VS4\n'],
       ['75:120', '75 - 120', 'P12_VS4\n'],
       ["'   '", 'Notappl', 'P12_VS4\n'],
       ['0', 'Total', 'P12_VS4'],
       ['1', 'Male', 'P11_VS1\n'],
       ['2', 'Female', 'P11_VS1\n'],
       ['3', 'Intersex', 'P11_VS1\n'],
       ['0', 'Total', 'P11_VS1']], dtype='<U32')

In [10]:
area_varname = [dimension[0]] * len(area_codes)


In [11]:
b = np.column_stack((area_codes, area_names, area_varname))
print(len(b))

393


In [12]:
c = np.concatenate((b, a), axis=0)

codes = pd.DataFrame(c)

codes.columns =  header

codes

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
404,0,Total,P12_VS4
405,1,Male,P11_VS1\n
406,2,Female,P11_VS1\n
407,3,Intersex,P11_VS1\n


In [13]:
codes['cod_list'].replace(regex=True,inplace=True,to_replace=r'\n',value=r'')
codes

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
404,0,Total,P12_VS4
405,1,Male,P11_VS1
406,2,Female,P11_VS1
407,3,Intersex,P11_VS1


In [14]:
all_codes = []
all_codes.append(area_codes)

for i, dim in enumerate(dimension[1:]):
    #print(i, dim)
    arr_cod = []
    for arr in all_values[i+1]:

        d =  ''
        result = codes.loc[(codes['cod_list'] == dim) & (codes['desc'] == arr)]
        try:
            d = result['cod'].values[0]
            arr_cod.append(d)
        except:
            pass
    print(len(arr_cod))
    all_codes.append(arr_cod)
        




12
4


In [15]:
all_codes

[['',
  '01',
  '0101',
  '0102',
  '0103',
  '0104',
  '0105',
  '0106',
  '02',
  '0201',
  '0202',
  '0203',
  '0204',
  '0205',
  '03',
  '0301',
  '0302',
  '0303',
  '0304',
  '0305',
  '0306',
  '0307',
  '0308',
  '0309',
  '04',
  '0401',
  '0402',
  '0403',
  '05',
  '0501',
  '0502',
  '06',
  '0601',
  '0602',
  '0603',
  '0604',
  '07',
  '0701',
  '0702',
  '0703',
  '0704',
  '0705',
  '0706',
  '0707',
  '08',
  '0801',
  '0802',
  '0803',
  '0804',
  '0805',
  '0806',
  '0807',
  '0808',
  '09',
  '0901',
  '0902',
  '0903',
  '0904',
  '0905',
  '0906',
  '0907',
  '10',
  '1001',
  '1002',
  '1003',
  '1004',
  '1005',
  '1006',
  '1007',
  '11',
  '1101',
  '1102',
  '1103',
  '12',
  '1201',
  '1202',
  '1203',
  '1204',
  '1205',
  '1206',
  '1207',
  '1208',
  '1209',
  '1210',
  '1211',
  '1212',
  '1213',
  '13',
  '1301',
  '1302',
  '1303',
  '1304',
  '1305',
  '1306',
  '14',
  '1401',
  '1402',
  '1403',
  '1404',
  '1405',
  '1406',
  '15',
  '1501',
  '1

In [16]:
result = list(itertools.product(*all_codes))

print(len(result))

18864


In [17]:
print(result)

[('', '0', '0'), ('', '0', '1'), ('', '0', '2'), ('', '0', '3'), ('', '3:5', '0'), ('', '3:5', '1'), ('', '3:5', '2'), ('', '3:5', '3'), ('', '6:9', '0'), ('', '6:9', '1'), ('', '6:9', '2'), ('', '6:9', '3'), ('', '10:14', '0'), ('', '10:14', '1'), ('', '10:14', '2'), ('', '10:14', '3'), ('', '15:24', '0'), ('', '15:24', '1'), ('', '15:24', '2'), ('', '15:24', '3'), ('', '25:34', '0'), ('', '25:34', '1'), ('', '25:34', '2'), ('', '25:34', '3'), ('', '35:44', '0'), ('', '35:44', '1'), ('', '35:44', '2'), ('', '35:44', '3'), ('', '45:54', '0'), ('', '45:54', '1'), ('', '45:54', '2'), ('', '45:54', '3'), ('', '55:64', '0'), ('', '55:64', '1'), ('', '55:64', '2'), ('', '55:64', '3'), ('', '65:74', '0'), ('', '65:74', '1'), ('', '65:74', '2'), ('', '65:74', '3'), ('', '75:120', '0'), ('', '75:120', '1'), ('', '75:120', '2'), ('', '75:120', '3'), ('', "'   '", '0'), ('', "'   '", '1'), ('', "'   '", '2'), ('', "'   '", '3'), ('01', '0', '0'), ('01', '0', '1'), ('01', '0', '2'), ('01', '0', '

In [18]:
df = pd.DataFrame(result)
df.columns = dimension
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18864 entries, 0 to 18863
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   SYSTEM_TOTAL  18864 non-null  object
 1   P12_VS4       18864 non-null  object
 2   P11_VS1       18864 non-null  object
dtypes: object(3)
memory usage: 442.2+ KB


In [19]:
len(all_rows)


18864

In [20]:
df['VALUES'] = all_rows
df['VALUES'] = df['VALUES'].str.replace(',000000000', '')
df

,SYSTEM_TOTAL,P12_VS4,P11_VS1,VALUES
0,,0,0,4373874
1,,0,1,2156668
2,,0,2,2217066
3,,0,3,140
4,,3:5,0,374139
...,...,...,...,...
18859,4711,75:120,3,
18860,4711,' ',0,
18861,4711,' ',1,
18862,4711,' ',2,


In [21]:
codes

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
404,0,Total,P12_VS4
405,1,Male,P11_VS1
406,2,Female,P11_VS1
407,3,Intersex,P11_VS1


In [22]:
d = codes.loc[codes['cod_list']=='SYSTEM_TOTAL']
d

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
388,4707,Makadara,SYSTEM_TOTAL
389,4708,Mathare,SYSTEM_TOTAL
390,4709,Njiru,SYSTEM_TOTAL
391,4710,Starehe,SYSTEM_TOTAL


In [23]:
for cod in dimension:
    
    cod_col = 'DESC_'+cod
    print(cod)
    d = codes.loc[codes['cod_list']==cod]
    df = pd.merge(df, d, how='inner', left_on=cod, right_on='cod')
    df = df.drop('cod', axis=1)
    df = df.drop('cod_list', axis=1)
    df.rename(columns={'desc': cod_col}, inplace=True) 
df

SYSTEM_TOTAL
P12_VS4
P11_VS1


,SYSTEM_TOTAL,P12_VS4,P11_VS1,VALUES,DESC_SYSTEM_TOTAL,DESC_P12_VS4,DESC_P11_VS1
0,,0,0,4373874,Kenya,Total,Total
1,01,0,0,109905,Mombasa,Total,Total
2,0101,0,0,12099,Changamwe,Total,Total
3,0102,0,0,15159,Jomvu,Total,Total
4,0103,0,0,26184,Kisauni,Total,Total
...,...,...,...,...,...,...,...
18859,4707,' ',3,,Makadara,Notappl,Intersex
18860,4708,' ',3,,Mathare,Notappl,Intersex
18861,4709,' ',3,,Njiru,Notappl,Intersex
18862,4710,' ',3,,Starehe,Notappl,Intersex


In [25]:
df.to_csv('./newout02.csv', index=True, index_label='id')